In [35]:
import pandas as pd
import numpy as np
import struct
import re

In [36]:
# file = pd.read_csv("pingtestdecode.txt", sep='-')
file = open("test3.txt",'r')
# firstCol = np.asarray(file[0])
# print(firstCol)

a = file.read()

a.find('7e')

# get indices
indices = [m.start() for m in re.finditer('7e', a)]

packet_list = []
for index in range(len(indices) - 1):
    packet = a[indices[index]:indices[index+1] + 2]
    b = packet.split(sep='-')
    for i in range(len(b)):
        b[i]  = int('0x' + b[i],16)
    packet_list.append(b)
    
packet_list[0]

[126, 47, 1, 240, 0, 0, 0, 0, 0, 0, 176, 4, 231, 231, 126]

In [37]:
def parse_ownship_report(data):
    if len(data) != 28:
        raise ValueError("Invalid data length for Ownship Report. Expected 24 bytes.")
    
    parsed_data = {
        "Message ID": data[0],
        "Traffic Alert Status": (data[1] & 0xF0) >> 4,
        "Address Type": data[1] & 0x0F,
        "Participant Address": data[2:5].hex(),
        "Latitude": int.from_bytes(data[5:8], byteorder='big', signed=True) * (180 / (2 ** 23)),
        "Longitude": int.from_bytes(data[8:11], byteorder='big', signed=True) * (180 / (2 ** 23)),
        "Altitude": (((data[11] << 8) | data[12]) >> 4) * 25 - 1000,
        "Miscellaneous Indicators": data[12] & 0x0F,
        "NIC": (data[13] & 0xF0) >> 4,
        "NACp": data[13] & 0x0F,
        "Horizontal Velocity": (data[14] << 4) | (data[15] >> 4),
        "Vertical Velocity": ((data[15] & 0x0F) << 8) | data[16],
        "Track/Heading": data[17],
        "Emitter Category": data[18],
        "Flight Identification": data[19:25].decode('ascii').strip(),
        # "Emergency/Priority Code": (data[23] & 0xF0) >> 4,
        # "Reserved": data[23] & 0x0F
    }
    
    return parsed_data


def parse_geometric_altitude(data):
    
    parsed_data = {
        'Message ID': data[0],
        'Geometric altitude (feet)': int.from_bytes(data[1:3], byteorder='big', signed=True) * 5 
    }
    return parsed_data

def parse_gnss_data(data):
    # if len(data) != 49:
        # raise ValueError("Invalid data length for GNSS Data message. Expected 49 bytes.")
    
    parsed_data = {
        "Message ID": data[0],
        "Message Version": data[1],
        "UTC Time": int.from_bytes(data[2:6], byteorder='big'),
        "Latitude": int.from_bytes(data[6:10], byteorder='big', signed=True) / 1e7,
        "Longitude": int.from_bytes(data[10:14], byteorder='big', signed=True) / 1e7,
        "Altitude": int.from_bytes(data[14:18], byteorder='big', signed=True) / 1e3,
        "Horizontal Protection Level (HPL)": int.from_bytes(data[18:22], byteorder='big') / 1e3,
        "Vertical Protection Level (VPL)": int.from_bytes(data[22:26], byteorder='big') / 1e2,
        "Horizontal Figure of Merit (HFOM)": int.from_bytes(data[26:30], byteorder='big') / 1e3,
        "Vertical Figure of Merit (VFOM)": int.from_bytes(data[30:32], byteorder='big') / 1e2,
        "Horizontal Velocity Figure of Merit (HVFOM)": int.from_bytes(data[32:34], byteorder='big') / 1e3,
        "Vertical Velocity Figure of Merit (VVFOM)": int.from_bytes(data[34:36], byteorder='big') / 1e3,
        "GNSS Vertical Speed": int.from_bytes(data[36:38], byteorder='big', signed=True) / 1e2,
        "North-South Velocity": int.from_bytes(data[38:40], byteorder='big', signed=True) / 1e1,
        "East-West Velocity": int.from_bytes(data[40:42], byteorder='big', signed=True) / 1e1,
        # "GNSS Fix Quality": data[42],
        # "Navigation State Indicator Flags": data[43],
        # "Number of Satellites": data[44]
    }
    
    return parsed_data

def parse_transponder_status(data):
    # if len(data) != 16:
        # raise ValueError("Invalid data length for Transponder Status message. Expected 16 bytes.")
    
    parsed_data = {
        "Message ID": data[0],
        "Message Version": data[1],
        "1090ES TX Enabled": (data[2] & 0x80) >> 7,
        # "Mode S Reply Enabled": (data[2] & 0x40) >> 6,
        # "Mode C Reply Enabled": (data[2] & 0x20) >> 5,
        # "Mode A Reply Enabled": (data[2] & 0x10) >> 4,
        "IDENT Button Active": (data[2] & 0x08) >> 3,
        # "Mode A Interrogation Replies per Second": int.from_bytes(data[3:5], byteorder='big'),
        # "Mode C Interrogation Replies per Second": int.from_bytes(data[5:7], byteorder='big'),
        # "Mode S Interrogation Replies per Second": int.from_bytes(data[7:9], byteorder='big'),
        # "Mode A Squawk Code": int.from_bytes(data[9:11], byteorder='big'),
        # "NACp": (data[11] & 0xF0) >> 4,
        # "NIC": data[11] & 0x0F,
        # "Board Temperature (°C)": data[12]
    }
    
    return parsed_data

def parse_barometer_sensor(data):
    if len(data) != 12:
        raise ValueError("Invalid data length for Barometer Sensor message. Expected 12 bytes.")
    
    parsed_data = {
        "Message ID": data[0],
        "Sensor Type": data[1],
        "Barometric Pressure (mbar)": int.from_bytes(data[2:6], byteorder='big') / 100,
        "Barometric Pressure Altitude (mm)": int.from_bytes(data[6:10], byteorder='big', signed=True),
        "Barometric Sensor Temperature (°C)": int.from_bytes(data[10:12], byteorder='big', signed=True) / 100
    }
    
    # Check for invalid values
    if parsed_data["Barometric Pressure Altitude (mm)"] == 0xFFFFFFFF:
        parsed_data["Barometric Pressure Altitude (mm)"] = "Invalid"
    if parsed_data["Barometric Sensor Temperature (°C)"] == 0xFFFF / 100:
        parsed_data["Barometric Sensor Temperature (°C)"] = "Invalid"
    
    return parsed_data





In [38]:
# CRC-CCITT calculation function based on the ICD
Crc16Table = [0] * 256

def crc_init():
    for i in range(256):
        crc = (i << 8)
        for bitctr in range(8):
            crc = (crc << 1) ^ (0x1021 if (crc & 0x8000) else 0)
        Crc16Table[i] = crc & 0xFFFF

def crc_compute(data):
    crc = 0
    for byte in data:
        crc = (Crc16Table[(crc >> 8) ^ byte] ^ (crc << 8)) & 0xFFFF
    return crc

# Unescape the control bytes
def unescape_payload(payload):
    unescaped = bytearray()
    i = 0
    while i < len(payload):
        if payload[i] == 0x7D:  # Control escape
            i += 1
            unescaped.append(payload[i] ^ 0x20)  # XOR next byte with 0x20
        else:
            unescaped.append(payload[i])
        i += 1
    return unescaped

# Decode the packet based on message ID
def decode_packet(packet):
    if len(packet) < 3:
        return "Invalid packet length"

    # Extract message ID and version (assume message ID is 1 byte, adjust as needed)
    msg_id = packet[0]
    version = packet[1]

    # Example: Handling Heartbeat message (Msg ID = 0x00)
    if msg_id == 0x00:  # Heartbeat
        gnss_valid = (packet[2] >> 7) & 0x01
        maintenance_req = (packet[2] >> 6) & 0x01
        ident_active = (packet[2] >> 5) & 0x01
        initialized = packet[2] & 0x01

        return {
            "Message ID": msg_id,
            "GNSS Valid": gnss_valid,
            "Maintenance Required": maintenance_req,
            "IDENT Active": ident_active,
            "Device Initialized": initialized
        }

    if msg_id == 0x0A:
        # print(packet,len(packet))
        return parse_ownship_report(packet)
    
    if msg_id == 0x0B:
        return parse_geometric_altitude(packet)

    if msg_id == 0x2E:
        # print('here')
        return parse_gnss_data(packet)
    
    if msg_id == 0x2F:
        return parse_transponder_status(packet)
    
    if msg_id == 0x28:
        return parse_barometer_sensor(packet)
    
    # Add handling for more messages (e.g., Ownship, GNSS Data) using the format from the ICD
    return msg_id, None

# Process the input packet
def process_packet(data):
    if data[0] != 0x7E or data[-1] != 0x7E:
        return "Invalid frame flags"
    
    # Remove framing flags (0x7E at start and end)
    packet = data[1:-1]

    # Unescape payload
    packet = unescape_payload(packet)

    # Verify CRC
    payload, crc_received = packet[:-2], packet[-2:]
    crc_computed = crc_compute(payload)
    crc_received = struct.unpack('>H', crc_received)[0]

    # print(crc_computed, crc_received)
    
    # if crc_computed != crc_received:
    #     return "CRC check failed"

    # Decode payload
    return decode_packet(payload)

# Test the script with a sample packet
# sample_packet = bytearray([0x7E, 0x80, 0x60, 0x18, 0x80, 0x80, 0x80, 0xFE, 0xFF, 0xFF, 0xFF, 0xFB, 0x30, 0x18, 0x7B, 0xFC, 0x7E])  # Example with escaped data
# sample_packet = bytearray(packet_list[6])
crc_init()  # Initialize CRC table
# result = process_packet(sample_packet)
# print(result)
for i,packet in enumerate(packet_list):
    try:
        sample_packet = bytearray(packet)
        # print(packet)
        result = process_packet(sample_packet)
        print(result)
        
    except:
        # Exception
        # print('Error')
        pass

{'Message ID': 47, 'Message Version': 1, '1090ES TX Enabled': 1, 'IDENT Button Active': 0}
{'Message ID': 40, 'Sensor Type': 1, 'Barometric Pressure (mbar)': 23411427.84, 'Barometric Pressure Altitude (mm)': 1410662400, 'Barometric Sensor Temperature (°C)': -17.81}
{'Message ID': 0, 'GNSS Valid': 0, 'Maintenance Required': 0, 'IDENT Active': 0, 'Device Initialized': 0}
(37, None)
{'Message ID': 10, 'Traffic Alert Status': 0, 'Address Type': 0, 'Participant Address': '000001', 'Latitude': 45.506486892700195, 'Longitude': -73.5764479637146, 'Altitude': 0, 'Miscellaneous Indicators': 9, 'NIC': 3, 'NACp': 6, 'Horizontal Velocity': 1, 'Vertical Velocity': 30, 'Track/Heading': 211, 'Emitter Category': 0, 'Flight Identification': 'VA2ETD'}
{'Message ID': 11, 'Geometric altitude (feet)': 1385}
{'Message ID': 47, 'Message Version': 1, '1090ES TX Enabled': 1, 'IDENT Button Active': 0}
{'Message ID': 40, 'Sensor Type': 1, 'Barometric Pressure (mbar)': 22740339.2, 'Barometric Pressure Altitude (mm

In [39]:
# data = bytearray(b'\n\x00\x00\x00\x01 \\\t\xcb\xad\xed\x02\xc9\x89\x00\x00\x00\x00\x00VA2ETD  \x00')
# data = bytearray(b'\n\x00\x00\x00\x01 \\\t\xcb\xad\xec\x02\xc9\x89\x00\x00\x00\x00\x00VA2ETD  \x00')
# print(len(data))

# parsed_data = parse_ownship_report(data)


# print(parsed_data)
